In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# @Time    : 2022/11/22 10:36
# @Author  : Wang Yujia
# @File    : auction_features_encoding_demo.ipynb
# @Description : Demo 为auction提取contextual features

# 1. preparations
## 1.1 全局设置


In [4]:

data_key_path = "../data/targets/target_datakey.csv"
prod_id_path = "../data/prod_id.csv"

# output path
prod_embedding_output_path = "../data/prod_embedding.csv"

import sys
import pandas as pd
import numpy as np
from pprint import pprint
import torch
from transformers import BertModel
from tokenizers import Tokenizer
from visdom import Visdom
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
import gzip
import csv
import os
import random
import logging
from sklearn.decomposition import PCA
from sentence_transformers import SentenceTransformer, util, InputExample,evaluation,models

from tokenizers.models import BPE
import tokenizers
# from tokenizers import BertWordPieceTokenizer

print(sys.path)
device = 'cuda'
print(torch.cuda.is_available())

['D:\\PyCharm\\plugins\\python\\helpers-pro\\jupyter_debug', 'D:\\PyCharm\\plugins\\python\\helpers\\pydev', 'D:\\Desktop\\PROJ\\PAProj\\data_handler', 'D:\\Desktop\\PROJ\\PAProj', 'D:\\Anaconda\\python39.zip', 'D:\\Anaconda\\DLLs', 'D:\\Anaconda\\lib', 'D:\\Anaconda', '', 'D:\\Anaconda\\Lib\\site-packages', 'D:\\Anaconda\\Lib\\site-packages\\win32', 'D:\\Anaconda\\Lib\\site-packages\\win32\\lib', 'D:\\Anaconda\\Lib\\site-packages\\Pythonwin', 'D:\\Anaconda\\Lib\\site-packages\\IPython\\extensions', 'C:\\Users\\Administrator\\.ipython']
True


## 1.2 读取data

In [126]:
data_key = pd.read_csv(data_key_path,encoding="utf-8")
prod_id_all = pd.read_csv(prod_id_path,encoding="utf-8")

prod_id = prod_id_all[ prod_id_all['id'].isin(data_key['id']) ]
prod_id.reset_index(drop=True,inplace=True)
print(prod_id.head(10))
print(prod_id.shape)

   id                                           desc
0   0  Sony Ericsson S500i Unlocked Mysterious Green
1   1               PSP Slim & Lite Sony Piano Black
2   2     iPod Touch Apple 8GB with Software Upgrade
3   3      Logitech Cordless Wave Keyboard and Mouse
4   4   Apple Macbook Air 1.6GHz Core 2 Duo Notebook
5   5                     SanDisk Cruzer Contour 4GB
6   6           Mario Kart with Wheel (Nintendo Wii)
7   7      PS3 | Playstation 3 Sony Console 40GB HDD
8   8                    DS | Nintendo DS Lite White
9  11            Corsair Voyager Mini 4 GB USB Flash
Done


## 1.3 读取model
1.

In [5]:
# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[CLS] 标志放在第一个句子的首位，经过 BERT 得到的的表征向量 C 可以用于后续的分类任务。
[SEP] 标志用于分开两个输入句子，例如输入句子 A 和 B，要在句子 A，B 后面增加 [SEP] 标志。
[UNK] 标志指的是未知字符
[MASK] 标志用于遮盖句子中的一些单词，将单词用 [MASK] 遮盖之后，再利用 BERT 输出的 [MASK] 向量预测单词是什么。

# 2. Tokenizer
## 2.1 使用预训练的分词器
> http://t.csdn.cn/nKArk
1. 英文是subword的概念，例如将"unwanted"分解成[“un”, “##want”, “##ed”]
2. 只要有 tokenizer.json 文件就能直接用 `from_pretrained` 加载。
3. 输出包括：
    - ‘input_ids’：顾名思义，是单词在词典中的编码
    - ‘token_type_ids’， 区分两个句子的编码
    - ’overflowing_tokens’, 当指定最大长度时，溢出的单词
    - ‘num_truncated_tokens’, 溢出的token数量
    - ‘return_special_tokens_mask’，如果添加特殊token，则这是[0，1]的列表，其中0指定特殊添加的token，而1指定序列标记


In [31]:
desc_str = prod_id.loc[21,'desc']
desc_str_2 = prod_id.loc[22,'desc']
desc_str,desc_str_2

('DS | Nintendo DS Lite (Crimson Red and Black)',
 'DS | Nintendo DS Lite (Cobalt Blue and Black)')

In [8]:
tokenizer_1 = Tokenizer.from_pretrained("bert-base-uncased")
output = tokenizer_1.encode(desc_str)

print("分词结果：\n",output.tokens)
print("分词id：\n",output.ids)
print("词典大小：\n",tokenizer_1.get_vocab_size())

分词结果：
 ['[CLS]', 'sony', 'eric', '##sson', 's', '##500', '##i', 'unlocked', 'mysterious', 'green', '[SEP]']
分词id：
 [101, 8412, 4388, 7092, 1055, 29345, 2072, 14058, 8075, 2665, 102]
词典大小：
 30522


# 3. encoding
## 3.1 句子相似度: SentenceTransformer
1. 可以选的models 见：https://www.sbert.net/docs/pretrained_models.html
2. The `all-*` models where trained on all available training data (more than 1 billion training pairs) and are designed as general purpose models.
    - all-mpnet-base-v2最好，all-MiniLM-L6-v2比较快（5倍）


In [40]:
model_2 = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# util.pytorch_cos_sim(embedding_1, embedding_2)

### PCA降维

1. https://github.com/UKPLab/sentence-transformers/blob/master/examples/training/distillation/dimensionality_reduction.py

In [69]:
new_dimension = 303

logger = logging.getLogger(__name__)

In [72]:
######## Reduce the embedding dimensions ########

#We use AllNLI as a source of sentences to compute PCA
nli_dataset_path = 'datasets/AllNLI.tsv.gz'

#We use the STS benchmark dataset to see how much performance we loose by using the dimensionality reduction
sts_dataset_path = 'datasets/stsbenchmark.tsv.gz'

if not os.path.exists(nli_dataset_path):
    util.http_get('https://sbert.net/datasets/AllNLI.tsv.gz', nli_dataset_path)

if not os.path.exists(sts_dataset_path):
    util.http_get('https://sbert.net/datasets/stsbenchmark.tsv.gz', sts_dataset_path)

# We measure the performance of the original model
# and later we will measure the performance with the reduces dimension size
logger.info("Read STSbenchmark test dataset")
eval_examples = []
with gzip.open(sts_dataset_path, 'rt', encoding='utf8') as fIn:
    reader = csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_NONE)
    for row in reader:
        if row['split'] == 'test':
            score = float(row['score']) / 5.0 #Normalize score to range 0 ... 1
            eval_examples.append(InputExample(texts=[row['sentence1'], row['sentence2']], label=score))

# Evaluate the original model on the STS benchmark dataset
stsb_evaluator = evaluation.EmbeddingSimilarityEvaluator.from_input_examples(eval_examples, name='sts-benchmark-test')

logger.info("Original model performance:")
stsb_evaluator(model_2)

0.8203247283076371

In [73]:
#Read sentences from NLI dataset
nli_sentences = set()
with gzip.open(nli_dataset_path, 'rt', encoding='utf8') as fIn:
    reader = csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_NONE)
    for row in reader:
        nli_sentences.add(row['sentence1'])
        nli_sentences.add(row['sentence2'])

nli_sentences = list(nli_sentences)
random.shuffle(nli_sentences)
nli_sentences.__len__()

In [75]:
#To determine the PCA matrix, we need some example sentence embeddings.
#Here, we compute the embeddings for 20k random sentences from the AllNLI dataset
pca_train_sentences = nli_sentences[0:20000]
train_embeddings = model_2.encode(pca_train_sentences, convert_to_numpy=True)

#Compute PCA on the train embeddings matrix
pca = PCA(n_components=new_dimension)
pca.fit(train_embeddings)
pca_comp = np.asarray(pca.components_)
pca_comp.shape

(303, 384)

(303, 384)

In [81]:
# Add a dense layer to the model, so that it will produce directly embeddings with the new size
dense = models.Dense(in_features=model_2.get_sentence_embedding_dimension(), out_features=new_dimension, bias=False, activation_function=torch.nn.Identity())
dense.linear.weight = torch.nn.Parameter(torch.tensor(pca_comp))
model_2.add_module('dense', dense)

# Evaluate the model with the reduce embedding size
logger.info("Model with {} dimensions:".format(new_dimension))
stsb_evaluator(model_2)

0.8234394986740291

### model generate and save

In [82]:
prod_embedding = model_2.encode(list(prod_id.loc[:,'desc']), convert_to_numpy=True,device=device)
assert prod_embedding.shape[0] == prod_id.shape[0], "Wrong!"

In [122]:
prod_embedding_df = pd.DataFrame(prod_embedding)
prod_embedding_df['id'] = prod_id['id']
prod_embedding_df['desc'] = prod_id['desc']

prod_embedding_df.to_csv(prod_embedding_output_path,header=True,index=False,encoding="utf-8")
prod_embedding_df.head(10),prod_embedding_df.shape

,0,1,2,3,4,5,6,7,8,9,...,295,296,297,298,299,300,301,302,id,desc
0,0.116034,0.020119,-0.021310,-0.075286,0.016879,0.018455,-0.034276,0.031793,-0.057861,-0.049602,...,-0.077679,-0.027134,0.007310,-0.076596,-0.025088,0.018122,-0.032752,0.014568,0,Sony Ericsson S500i Unlocked Mysterious Green
1,0.061373,0.032199,0.011518,0.034196,-0.077273,0.059780,0.041093,0.074851,0.053483,0.116248,...,-0.049425,0.017499,0.006069,0.010160,0.003023,0.031978,-0.038282,0.011311,1,PSP Slim & Lite Sony Piano Black
2,-0.051620,-0.072511,0.111702,0.046107,-0.020478,0.150707,0.056321,0.018349,0.011857,-0.039793,...,-0.043603,0.015063,-0.011231,0.017059,-0.039363,-0.002873,-0.015840,-0.004983,2,iPod Touch Apple 8GB with Software Upgrade
3,-0.000960,0.025139,0.003293,0.072950,-0.010923,-0.078227,0.151397,0.171092,-0.007707,0.072944,...,-0.026399,-0.049374,-0.024551,0.010284,0.009512,0.006033,0.008652,0.042826,3,Logitech Cordless Wave Keyboard and Mouse
4,0.053038,0.071515,0.017683,-0.006201,-0.022760,-0.008116,0.085413,0.048034,0.087793,-0.058904,...,-0.075914,-0.008973,0.043354,0.005005,0.021273,0.029586,0.014097,0.034332,4,Apple Macbook Air 1.6GHz Core 2 Duo Notebook
5,0.012295,-0.014781,0.060551,-0.009491,0.094940,0.034042,-0.050308,-0.018858,-0.017407,0.004311,...,-0.053434,0.036242,0.020129,0.005436,-0.030297,0.013453,-0.025003,-0.004230,5,SanDisk Cruzer Contour 4GB
6,0.090535,0.037109,0.043333,0.141303,0.034144,0.002742,0.017636,-0.103622,-0.102304,-0.016737,...,-0.000215,0.024440,-0.007869,0.026648,-0.056101,-0.030093,0.004264,-0.000276,6,Mario Kart with Wheel (Nintendo Wii)
7,-0.011322,0.008502,0.006862,-0.057471,0.035309,0.021279,0.013266,-0.028090,0.056078,-0.001076,...,-0.013573,0.027576,0.004757,-0.026771,-0.033586,-0.014026,0.036300,0.019918,7,PS3 | Playstation 3 Sony Console 40GB HDD
8,0.003335,0.086421,0.009364,0.058529,-0.030463,-0.028964,-0.044886,-0.060023,-0.061517,-0.016009,...,-0.045395,-0.008474,0.006412,0.007268,-0.034747,0.006816,-0.021698,-0.010173,8,DS | Nintendo DS Lite White
9,-0.034880,0.081387,0.151240,0.008757,-0.003367,-0.042134,0.031805,0.012283,-0.030045,-0.035092,...,-0.006119,0.043008,0.000976,-0.034087,-0.017426,0.016793,-0.006562,-0.004813,11,Corsair Voyager Mini 4 GB USB Flash


### clustering
1. 先用tsne降维

In [5]:
LENTH=300      # 望得到的encoding维度=300 or 4? 取决于用途
prod_embedding_small_path = "../data/small_prod_embedding_"+str(LENTH)+".csv"
prod_embedding_large_path = "E:\DATA\large_dta\large_prod_embedding_"+str(LENTH)+".csv"

In [6]:
prod_embedding_small = pd.read_csv(prod_embedding_small_path,encoding="utf-8")
prod_embedding_large = pd.read_csv(prod_embedding_large_path,encoding="utf-8")


In [7]:
num_clusters = 4

In [6]:
# prod_embedding_path = "../data/prod_embedding_100.csv"
# prod_embedding_100 = pd.read_csv(prod_embedding_path,encoding="utf-8")
# prod_embedding = prod_embedding_100[:,:100]

# perplexity: 默认为30，数据集越大，需要参数值越大，建议值位5-50 , n_components=2 默认为2，嵌入空间的维度（嵌入空间的意思就是结果空间）
tsne_model = TSNE(perplexity=10, n_components=2, init='pca', n_iter=250, random_state=23)
new_values = tsne_model.fit_transform(prod_embedding)  # 新坐标
x = []
y = []
for value in new_values:
    x.append(value[0])
    y.append(value[1])


clustering_model = KMeans(n_clusters=num_clusters)
clustering_model.fit(prod_embedding)
cluster_assignment = clustering_model.labels_   # cluster 归属

# 长度为num_clusters的list，每个元素也是一个list
clustered_sentences = [[] for i in range(num_clusters)]

# cluster_assignment是array
# enumerate可以同时获得索引和值
# clustered_sentences记录了num_clusters个cluster里面的desc
for sentence_id, cluster_id in enumerate(cluster_assignment):
    clustered_sentences[cluster_id].append( list(prod_id.loc[:,'desc'])[sentence_id] )


InvalidIndexError: (slice(None, None, None), slice(None, 100, None))

In [125]:
# np.save("new_value",new_values)
np.save("cluster_assignment",clustering_model.labels_)


In [85]:
viz = Visdom()
viz.scatter(X=new_values, Y=cluster_assignment+1,
        opts=dict(
            colormap='Jet',
            markersymbol="dot",  # "x"
            markersize=10,
            legend = ['0','1','2','3','4'],
            textlabels= list(prod_id.loc[:,'desc'])
        ))


Setting up a new session...


'window_3b4d30aea14cdc'

In [123]:
viz.save(["main"])

'["main"]'

## 3.2 使用bert的encoder
1. 直接把分词后的`id`当做input，然后得到输出。
2.

True


In [10]:
model.eval() # 将模型设为验证模式
input_ids = torch.tensor([output.ids]) # 添加batch维度并转化为tensor
# token_type_ids = torch.tensor([output['token_type_ids']])

tokens_tensor = input_ids.to(device)
# segments_tensors = token_type_ids.to(device)
model.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

Bert的输出：
    - sequence_output：输出序列,大小为`(1,11,768)`其中11是因为`desc`长度为11
    - pooler_output,对输出序列进行pool操作的结果
    - (hidden_states),隐藏层状态(包括Embedding层)，取决于modelconfig中output_hidden_states
    - (attentions), 注意力层，取决于参数中output_attentions

In [26]:
# 进行编码

with torch.no_grad():
    # See the models docstrings for the detail of the inputs
    # , token_type_ids=segments_tensors
    sequence_output= model(tokens_tensor)
    # Transformers models always output tuples.
    # See the models docstrings for the detail of all the outputs
    # In our case, the first element is the hidden state of the last layer of the Bert model
    print(sequence_output)
    sequence_output
# 得到最终的编码结果encoded_layers
pooled_output

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.4594, -0.2017, -0.0968,  ..., -0.2735,  0.3161,  0.3787],
         [-0.3372,  0.0911,  0.1787,  ...,  0.0088,  1.0508, -0.3864],
         [-0.0296,  0.5422,  0.9316,  ...,  0.4370,  0.8096, -0.7587],
         ...,
         [-0.1785, -0.2206,  0.1787,  ..., -0.0660, -0.0233,  0.2102],
         [-0.3025, -0.4273,  0.1322,  ...,  0.0447,  0.2715, -0.3983],
         [ 0.7027, -0.0926, -0.2899,  ...,  0.1964, -0.6339, -0.2348]]],
       device='cuda:0'), pooler_output=tensor([[-7.8197e-01, -3.8046e-01, -4.4998e-01,  7.0813e-01,  2.3026e-01,
         -1.4321e-01,  7.7712e-01,  3.1890e-01, -1.0563e-01, -9.9996e-01,
          9.4048e-02,  4.0133e-01,  9.4646e-01,  1.0262e-01,  8.2900e-01,
         -3.7846e-01,  1.5961e-01, -4.5966e-01,  3.4073e-01, -3.7634e-01,
          6.4000e-01,  9.9932e-01,  2.1653e-01,  2.0881e-01,  4.3850e-01,
          6.2764e-01, -4.1283e-01,  8.4835e-01,  9.3581e-01,  7.0680e-01,
         -4.

'pooler_output'